## Import Libraries

In [1]:
import time
start = time.time()
import numpy as np
import pandas as pd
from collections import defaultdict

from nltk import pos_tag
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm #SVR is in SVM
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.svm import SVR
from sklearn import linear_model
from sklearn.metrics import classification_report
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import train_test_split

## Function to get Naive Bayes QWK score

In [2]:
def get_nb_qwk(X_train, X_test, y_train, y_test):
    X_trainNB = X_train
    y_trainNB = y_train
    X_testNB = X_test
    y_testNB = y_test
    model_nbl = naive_bayes.MultinomialNB()
    model_nbl.fit(X_trainNB, y_trainNB.ravel())
    
    y_predNBL = model_nbl.predict(X_testNB)
    return cohen_kappa_score(y_test, y_predNBL, weights="quadratic"), y_predNBL

## Function to get SVM QWK score

In [3]:
def get_svm_qwk(X_train, X_test, y_train, y_test):
    sc_Xsvm = StandardScaler()
    sc_ysvm = StandardScaler()
    X_trainSVM = sc_Xsvm.fit_transform(X_train)
    y_trainSVM = sc_ysvm.fit_transform(y_train)
    X_testSVM = sc_Xsvm.transform(X_test)
    y_testSVM = sc_ysvm.transform(y_test)
    
    model_svml = SVR(kernel='rbf', gamma='auto', verbose=True)
    model_svml.fit(X_trainSVM,y_trainSVM.ravel())
    
    y_predSVML = model_svml.predict(X_testSVM)
    y_predSVML = sc_ysvm.inverse_transform(y_predSVML).round()
    return cohen_kappa_score(y_test, y_predSVML, weights="quadratic"), y_predSVML

## Function to get BLRR QWK score

In [4]:
def get_blrr_qwk(X_train, X_test, y_train, y_test):
    sc_Xblrr = StandardScaler()
    sc_yblrr = StandardScaler()
    X_trainBLRR = sc_Xblrr.fit_transform(X_train)
    y_trainBLRR = sc_yblrr.fit_transform(y_train)
    X_testBLRR = sc_Xblrr.transform(X_test)
    y_testBLRR = sc_yblrr.transform(y_test)
    model_blrrl = linear_model.BayesianRidge()
    model_blrrl.fit(X_trainBLRR, y_trainBLRR.ravel())
    y_predBLRRL = model_blrrl.predict(X_testBLRR)
    y_predBLRRL = sc_yblrr.inverse_transform(y_predBLRRL).round()
    return cohen_kappa_score(y_test, y_predBLRRL, weights="quadratic"), y_predBLRRL

## Function to get Ensemble QWK score

In [5]:
def get_ensemble_qwk(y_test, y_predNBL, y_predSVML, y_predBLRRL):
    actual = pd.Series(y_test.ravel())
    predNBL = pd.Series(y_predNBL)
    predSVML = pd.Series(y_predSVML)
    predBLRRL = pd.Series(y_predBLRRL)

    data = {"Actual": actual,
            "NBL": predNBL, 
            "SVML": predSVML, 
            "BLRRL": predBLRRL} 
    results = pd.concat(data, axis=1)
    results['Ensemble'] = np.where(
                            (results['NBL'] == results['BLRRL']) |
                            (results['NBL'] == results['SVML']),
                            results['NBL'],
                            results['BLRRL']
                        )
    
    return cohen_kappa_score(y_test, results['Ensemble'], weights="quadratic"),results['Ensemble']

## Function to get features from dataset

In [6]:
def get_train_test(dataset,featuretype,single_or_exclude):
    if featuretype == 'length' and single_or_exclude == 0:
        X = dataset.iloc[:,1:7].values.astype(float)
        y = dataset.iloc[:,15].values.astype(float)
        y = np.array(y).reshape(-1,1)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
        return X_train, X_test, y_train, y_test
    elif featuretype == 'pos' and single_or_exclude == 0:
        X = dataset.iloc[:,7:9].values.astype(float)
        y = dataset.iloc[:,15].values.astype(float)
        y = np.array(y).reshape(-1,1)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
        return X_train, X_test, y_train, y_test
    elif featuretype == 'bow' and single_or_exclude == 0:
        X = dataset.iloc[:,13:15].values.astype(float)
        y = dataset.iloc[:,15].values.astype(float)
        y = np.array(y).reshape(-1,1)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
        return X_train, X_test, y_train, y_test
    elif featuretype == 'prompt' and single_or_exclude == 0:
        X = dataset.iloc[:,9:13].values.astype(float)
        y = dataset.iloc[:,15].values.astype(float)
        y = np.array(y).reshape(-1,1)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
        return X_train, X_test, y_train, y_test
    elif featuretype == 'length' and single_or_exclude == 1:
        X = dataset.iloc[:,10:15].values.astype(float)
        y = dataset.iloc[:,15].values.astype(float)
        y = np.array(y).reshape(-1,1)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
        return X_train, X_test, y_train, y_test
    elif featuretype == 'pos' and single_or_exclude == 1:
        X = dataset.iloc[:,[1,2,3,4,5,6,9,10,11,12,13,14]].values.astype(float)
        y = dataset.iloc[:,15].values.astype(float)
        y = np.array(y).reshape(-1,1)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
        return X_train, X_test, y_train, y_test
    elif featuretype == 'bow' and single_or_exclude == 1:
        X = dataset.iloc[:,1:13].values.astype(float)
        y = dataset.iloc[:,15].values.astype(float)
        y = np.array(y).reshape(-1,1)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
        return X_train, X_test, y_train, y_test
    elif featuretype == 'prompt' and single_or_exclude == 1:
        X = dataset.iloc[:,[1,2,3,4,5,6,7,8,13,14]].values.astype(float)
        y = dataset.iloc[:,15].values.astype(float)
        y = np.array(y).reshape(-1,1)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
        return X_train, X_test, y_train, y_test
    elif featuretype == 'all':
        X = dataset.iloc[:,1:15].values.astype(float)
        y = dataset.iloc[:,15].values.astype(float)
        y = np.array(y).reshape(-1,1)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
        return X_train, X_test, y_train, y_test

## Function to get all QWK score

In [7]:
def get_qwk_all(Featuretype, X_train, X_test, y_train, y_test):
    # Getting the QWK scores for all methods
    nb_qwk, y_predNB = get_nb_qwk(X_train, X_test, y_train, y_test)
    svm_qwk, y_predSVM = get_svm_qwk(X_train, X_test, y_train, y_test)
    blrr_qwk, y_predBLRR = get_blrr_qwk(X_train, X_test, y_train, y_test)
    ensemble_qwk, y_predEnsemble = get_ensemble_qwk(y_test, y_predNB, y_predSVM, y_predBLRR)
    return Featuretype, nb_qwk, svm_qwk, blrr_qwk, ensemble_qwk

## Get qwk for single feature type 

In [8]:
dataset = pd.read_csv('maes_features.csv')
#length only
X_train, X_test, y_train, y_test = get_train_test(dataset,'length',0)
onlylength = get_qwk_all('Only Length',X_train, X_test, y_train, y_test )

#pos only
X_train, X_test, y_train, y_test = get_train_test(dataset,'pos',0)
onlypos = get_qwk_all('Only PoS',X_train, X_test, y_train, y_test )

#bow only
X_train, X_test, y_train, y_test = get_train_test(dataset,'bow',0)
onlybow = get_qwk_all('Only BoW',X_train, X_test, y_train, y_test )

#prompt only
X_train, X_test, y_train, y_test = get_train_test(dataset,'prompt',0)
onlyprompt = get_qwk_all('Only Prompt',X_train, X_test, y_train, y_test )

[LibSVM][LibSVM][LibSVM][LibSVM]

## Get qwk for excluding one feature

In [9]:
#excluding length
X_train, X_test, y_train, y_test = get_train_test(dataset,'length',1)
exclength = get_qwk_all('Excluding Length',X_train, X_test, y_train, y_test )

#excluding pos
X_train, X_test, y_train, y_test = get_train_test(dataset,'pos',1)
excpos = get_qwk_all('Excluding PoS',X_train, X_test, y_train, y_test )

#excluding bow
X_train, X_test, y_train, y_test = get_train_test(dataset,'bow',1)
excbow = get_qwk_all('Excluding BoW',X_train, X_test, y_train, y_test )

#excluding prompt
X_train, X_test, y_train, y_test = get_train_test(dataset,'prompt',1)
excprompt = get_qwk_all('Excluding Prompt',X_train, X_test, y_train, y_test )

[LibSVM][LibSVM][LibSVM][LibSVM]

## Making table for comparison

In [10]:
table = []
table.append(onlylength)
table.append(onlypos)
table.append(onlybow)
table.append(onlyprompt)
table.append(exclength)
table.append(excpos)
table.append(excbow)
table.append(excprompt)
#all features
X_train, X_test, y_train, y_test = get_train_test(dataset,'all',0)
table.append(get_qwk_all('All features',X_train, X_test, y_train, y_test ))

[LibSVM]

In [11]:
table_df = pd.DataFrame(table, columns=['Type', 'NB', 'SVM', 'BLRR', 'Ensemble'])
table_df

,Type,NB,SVM,BLRR,Ensemble
0,Only Length,0.504200,0.580145,0.603662,0.598689
1,Only PoS,0.014533,0.597624,0.564780,0.572198
2,Only BoW,0.000000,0.590307,0.573816,0.569452
3,Only Prompt,0.245569,0.569083,0.543177,0.542645
4,Excluding Length,0.443869,0.564516,0.601342,0.561465
5,Excluding PoS,0.510908,0.582942,0.617268,0.603770
6,Excluding BoW,0.545798,0.598798,0.604046,0.632497
7,Excluding Prompt,0.494043,0.636467,0.657317,0.656222
8,All features,0.517274,0.601138,0.626159,0.606082


### Final Conclusion - training model based on single type of feature
Below are a table conclude the result of model trained by single type of feature.


| Only Feature Used | Naive Bayes| SVM | BLRR | Ensemble | 
| --- | --- | --- | --- | --- | 
| Length | 0.504 | 0.580 | 0.604 | 0.599 | 
| PoS | 0.483 | 0.584 | 0.536 | 0.557 | 
| BoW | 0.0 | 0.590 | 0.574 | 0.569 | 
| Prompt | 0.246 | 0.569 | 0.543 | 0.542 | 
| All features | 0.517 | 0.601 | 0.626 | 0.606 |

Differences between all features and only features

| Only Features | Naive Bayes| SVM | BLRR | Ensemble | Total Diff (absolute value)  |
| --- | --- | --- | --- | --- | --- |
| Length | 0.013 | 0.021 | 0.022 | 0.007 | 0.063 |
| PoS | 0.034 | 0.017 | 0.090 | 0.049 | 0.190 |
| BoW | 0.517 | 0.011 | 0.052 | 0.037 | 0.617 |
| Prompt | 0.271 | 0.032 | 0.083 | 0.064 | 0.450 |

Higher in differences should mean lower in influence on the model.


`BoW` has no influence at all on Naive Bayes Model.

`Prompt` has the least influence on SVM and second least influence in BLRR.

`Length` has the most influences among all model.

### Final Conclusion - Influence
Below are a table conclude the result of model trained by excluding features


| Excluded Features | Naive Bayes| SVM | BLRR | Ensemble | 
| --- | --- | --- | --- | --- | 
| Length | 0.444 | 0.565 | 0.601 | 0.561 | 
| PoS | 0.511 | 0.583 | 0.617 | 0.604 | 
| BoW | 0.546 | 0.599 | 0.604 | 0.632 | 
| Prompt | 0.494 | 0.636 | 0.657 | 0.656 | 
| All features | 0.517 | 0.601 | 0.626 | 0.606 |


Differences between all features and excluded features

| Excluded Features | Naive Bayes| SVM | BLRR | Ensemble | Total Diff (absolute value) | Total Diff |
| --- | --- | --- | --- | --- | --- | --- |
| Length | 0.073 | 0.036 | 0.025 | 0.045 | 0.179 | 0.179 |
| PoS | 0.006 | 0.018 | 0.009 | 0.002 | 0.035 | 0.035 |
| BoW | -0.029 | 0.002 | 0.022 | -0.026 | 0.079 | -0.031 |
| Prompt | 0.023 | -0.035 | -0.031 | -0.050 | 0.139 | -0.093 |


Lower in differences should mean lower in influence on the model.

`Length` has the most influence on Naive Bayes, SVM Model.

`Prompt` has overfitting in SVM and BLRR model

`PoS` has the least influence among all model

`BoW` overfitting NB model